In [2]:
import sys
import pandas as pd
import numpy as np
import sklearn
import requests
import re
import time

In [3]:
#format URL, adding appropriate headers
def URL_format(url):
    http_flag = 0
    www_flag = 0
    new_url = ""
    
    if "http" not in url[:4]:
        http_flag = 1
    if "www" not in url[:11]:
        www_flag = 1

    if http_flag == 1:
        try:
            r = requests.get('https://'+ url, timeout=10)
            new_url = "https://"
        except:
            new_url = "http://"
            
        if www_flag == 1:
            new_url = new_url + "www."
            
        new_url = new_url+url
    else:
        if www_flag == 1:
            if url[:5] == "https":
                new_url = url[:8] + "www." + url[8:]
            elif url[:4] == "http":
                new_url = url[:7] + "www." + url[7:]
        else:
            new_url = url
    time.sleep(2)
    return new_url

#Length of URL
def URL_len(url):
    return len(url)

#Length of Hostname
def hostname_len(url):
    #assumes we start with ://www....
    matches = re.findall("://www.([\w\-\.]+)", str(url))
    result = 0
    if (len(matches) >= 1):
        result = len(matches[0])
    return result

#Length of path
def path_len(url):
    match_len = len(re.findall("://www.[\w\-\.]+([\/\w+]*)", str(url)))
    if (match_len > 0):
        return len(re.findall("://www.[\w\-\.]+([\/\w+]*)", str(url))[0])
    return match_len

#Length of first directory
def first_dir_len(url):
    match_len = len(re.findall("://www.[\w\-\.]+(\/\w+)", str(url)))
    if (match_len > 0):
        return len(re.findall("://www.[\w\-\.]+(\/\w+)", str(url))[0])
    return match_len
    
#Length of top level domain
def top_level_len(url):
    domains = re.findall("://www.([\w\-\.]+)", str(url))
    if len(domains) != 0:
        tld = domains[0].split('.')
        return len(tld[-1])
    return 0

#Count of '-'
def count_dash(url):
    dashes = 0
    for char in url:
        if char == '-':
            dashes += 1     
    return dashes

#Count of '@'
def count_at(url):
    at = 0
    for char in url:
        if char == '@':
            at += 1     
    return at

#Count of '?'
def count_q(url):
    q = 0
    for char in url:
        if char == '?':
            q += 1     
    return q

#Count of '%'
def count_percent(url):
    percent = 0
    for char in url:
        if char == '%':
            percent += 1     
    return percent

#Count of '.'
def count_dot(url):
    dot = 0
    for char in url:
        if char == '.':
            dot += 1     
    return dot

#Count of '='
def count_equal(url):
    eq = 0
    for char in url:
        if char == '=':
            eq += 1     
    return eq

#Count of ';'
def count_colon(url):
    colon = 0
    for char in url:
        if char == ';':
            colon += 1     
    return colon

#Count of 'www'
def count_www(url):
    www = 0
    www = len(re.findall("www", str(url)))
    return www

#Count of numbers
def count_numbers(url):
    numbers = sum(c.isdigit() for c in url)
    return numbers

#Count of letters
def count_letters(url):
    letters = sum(c.isalpha() for c in url)
    return letters

#Count of directories
def dir_count(url):
    dirs = 0
    for char in url:
        if char == '/':
            dirs = dirs + 1
    #assumes that it starts with ://www....
    return dirs - 2

#Count of single letter directories
def single_letter_dir(url):
    sldir = 0
    sldir = len(re.findall("\/[\w]\/", str(url)))
    return sldir
                
#Count of queries
def query_count(url):
    count = 0
    count = len(re.findall("\?\w+(&?\w+)*", str(url)))
    return count

#Ratio of uppercase to lowercase letters
def ratio_upper_lower(url):
    upperSum = 0
    lowerSum = 0
    for char in url:
        if char.isupper():
            upperSum = upperSum + 1
        if char.islower():
            lowerSum = lowerSum + 1
    # if url has 0 of one, this will return 0
    if lowerSum == 0:
        return upperSum
    return upperSum / lowerSum

#IP vs not
def is_ip(url):
    if len(re.findall("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}", str(url))) > 0:
        return 1
    else:
        return 0
    
# Shortened or not
def is_shortened(url):
    shortened = { "bit.ly", "tinyurl", "goo.gl", "ow.ly", "t.co", 
                 "tiny.cc", "bit.do", "shorte.st", "cutt.ly", "clkim"}
    
    if any(shrt in url for shrt in shortened):
        return 1
    else:
        return 0

#HTTP vs. HTTPS
#Returns 1 if HTTPS, 0 if HTTP
def is_https(url):
    if url[:5] == 'https':
        return 1
    elif url[:4] == 'http':
        return 0


In [4]:
#Read csv data using pandas
majestic_million=pd.read_csv('majestic_million.csv')
phishtank=pd.read_csv('phistank_verified_online_data.csv')

#Only take the first 10,000 values of the majestic million values
majestic_million=majestic_million.head(1000)

#Rows are store in res[] after being processed
res=[]

malicious=0
benign=1


In [5]:
for index,row in majestic_million.iterrows():
    #print(row.IDN_Domain)
    url = URL_format(row.IDN_Domain)
    print(index)
    print(url)

    url_res = URL_len(url)
    hostname_res = hostname_len(url)
    path_res = path_len(url)
    first_dir_res = first_dir_len(url)
    top_level_res = top_level_len(url)
    dash_res = count_dash(url)
    at_res = count_at(url)
    q_res = count_q(url)
    percent_res = count_percent(url)
    dot_res = count_dot(url)
    equal_res = count_equal(url)
    colon_res = count_colon(url)
    www_res = count_www(url)
    numbers_res = count_numbers(url)
    letters_res = count_letters(url)
    dir_res = dir_count(url)
    single_letter_res = single_letter_dir(url) 
    query_res = query_count(url)
    ratio_upper_lower_res = ratio_upper_lower(url)
    ip_res = is_ip(url)
    shortened_res = is_shortened(url)
    http_res = is_https(url)
    
    res.append([url,benign,url_res,hostname_res ,path_res,first_dir_res,top_level_res,dash_res,at_res,q_res,percent_res,dot_res,equal_res,colon_res,www_res,numbers_res,letters_res,dir_res,single_letter_res,query_res,ratio_upper_lower_res,ip_res,shortened_res,http_res])
  

0
https://www.google.com
1
https://www.facebook.com
2
https://www.youtube.com
3
https://www.twitter.com
4
https://www.instagram.com
5
https://www.linkedin.com
6
https://www.microsoft.com
7
https://www.apple.com
8
https://www.wikipedia.org
9
https://www.googletagmanager.com
10
https://www.youtu.be
11
https://www.en.wikipedia.org
12
https://www.plus.google.com
13
https://www.pinterest.com
14
https://www.play.google.com
15
https://www.vimeo.com
16
https://www.maps.google.com
17
https://www.adobe.com
18
https://www.goo.gl
19
https://www.wordpress.com
20
https://www.bit.ly
21
https://www.itunes.apple.com
22
https://www.github.com
23
https://www.wordpress.org
24
https://www.blogspot.com
25
https://www.docs.google.com
26
https://www.player.vimeo.com
27
https://www.amazon.com
28
https://www.mozilla.org
29
https://www.yahoo.com
30
https://www.tumblr.com
31
https://www.drive.google.com
32
https://www.support.google.com
33
https://www.apps.apple.com
34
https://www.flickr.com
35
https://www.europa

290
https://www.businesswire.com
291
https://www.yale.edu
292
https://www.aws.amazon.com
293
https://www.sciencedaily.com
294
https://www.hubspot.com
295
https://www.addons.mozilla.org
296
https://www.vox.com
297
https://www.netflix.com
298
https://www.psu.edu
299
https://www.fb.me
300
https://www.plesk.com
301
https://www.cisco.com
302
https://www.redhat.com
303
https://www.tripod.com
304
https://www.iso.org
305
https://www.tiktok.com
306
https://www.podcasts.apple.com
307
https://www.zdnet.com
308
https://www.books.google.com
309
https://www.gofundme.com
310
https://www.noaa.gov
311
https://www.disqus.com
312
https://www.wikihow.com
313
https://www.cbc.ca
314
https://www.state.gov
315
https://www.upenn.edu
316
https://www.intel.com
317
https://www.feeds.feedburner.com
318
https://www.vkontakte.ru
319
https://www.geocities.com
320
https://www.worldbank.org
321
http://www.cambridge.org
322
https://www.docs.microsoft.com
323
https://www.abc.net.au
324
https://www.guardian.co.uk
325
http

578
http://www.dot.gov
579
https://www.sec.gov
580
https://www.moodle.org
581
https://www.mystrikingly.com
582
https://www.shutterstock.com
583
https://www.gov.cn
584
http://www.sakura.ne.jp
585
https://www.mixcloud.com
586
https://www.matterport.com
587
https://www.scmp.com
588
https://www.apachefriends.org
589
https://www.login.microsoftonline.com
590
https://www.unicef.org
591
https://www.pan.baidu.com
592
https://www.ok.ru
593
https://www.openssl.org
594
https://www.news.com.au
595
https://www.jquery.com
596
https://www.drupal.org
597
https://www.photos.app.goo.gl
598
https://www.wp.me
599
https://www.thehill.com
600
https://www.namecheap.com
601
https://www.is.gd
602
https://www.news.google.com
603
https://www.readthedocs.io
604
https://www.kernel.org
605
https://www.icann.org
606
https://www.nhs.uk
607
https://www.hollywoodreporter.com
608
https://www.ru.wikipedia.org
609
http://www.go.th
610
https://www.weather.com
611
https://www.freebsd.org
612
https://www.getbootstrap.com
613

870
http://www.jiathis.com
871
http://www.nasdaq.com
872
https://www.duckduckgo.com
873
https://www.gamespot.com
874
https://www.tinypic.com
875
https://www.jimdofree.com
876
http://www.seattletimes.com
877
http://www.dedecms.com
878
https://www.medlineplus.gov
879
https://www.garmin.com
880
https://www.patch.com
881
https://www.sony.com
882
https://www.pbase.com
883
https://www.en.wiktionary.org
884
https://www.ibb.co
885
https://www.iheart.com
886
https://www.ucsb.edu
887
https://www.hyatt.com
888
https://www.snopes.com
889
https://www.diigo.com
890
https://www.speedtest.net
891
https://www.nsw.gov.au
892
https://www.lefigaro.fr
893
https://www.amazon.fr
894
https://www.download.cnet.com
895
https://www.zoho.com
896
https://www.ameblo.jp
897
https://www.jetpack.wordpress.com
898
https://www.wikidot.com
899
https://www.ibtimes.com
900
https://www.it.wikipedia.org
901
https://www.craigslist.org
902
https://www.1drv.ms
903
https://www.icloud.com
904
https://www.fifa.com
905
http://www.e

In [6]:
for index,row in phishtank.iterrows():
    #print("original: " + row.url)
    url = row.url
    #print("formatted: " + url)
    
    url_res = URL_len(url)
    hostname_res = hostname_len(url)
    path_res = path_len(url)
    first_dir_res = first_dir_len(url)
    top_level_res = top_level_len(url)
    dash_res = count_dash(url)
    at_res = count_at(url)
    q_res = count_q(url)
    percent_res = count_percent(url)
    dot_res = count_dot(url)
    equal_res = count_equal(url)
    colon_res = count_colon(url)
    www_res = count_www(url)
    numbers_res = count_numbers(url)
    letters_res = count_letters(url)
    dir_res = dir_count(url)
    single_letter_res = single_letter_dir(url) 
    query_res = query_count(url)
    ratio_upper_lower_res = ratio_upper_lower(url)
    ip_res = is_ip(url)
    shortened_res = is_shortened(url)
    http_res = is_https(url)
    
    res.append([url,malicious,url_res,hostname_res ,path_res,first_dir_res,top_level_res,dash_res,at_res,q_res,percent_res,dot_res,equal_res,colon_res,www_res,numbers_res,letters_res,dir_res,single_letter_res,query_res,ratio_upper_lower_res,ip_res,shortened_res,http_res])

result_column_names = ['url','malicious','url_res','hostname_res','path_res','first_dir_res','top_level_res','dash_res','at_res','q_res','percent_res','dot_res','equal_res','colon_res','www_res','numbers_res','letters_res','dir_res','single_letter_res','query_res','ratio_upper_lower_res','ip_res','shortened_res','http_res']

result = pd.DataFrame(res,columns=result_column_names)
result.to_csv('out.csv', sep=',',index=False)